# The lifecycle outlines the major stages that projects typically execute. There are six stages to build a model


Business Understanding 

Data Understanding

Data Preparation

Evaluation

Modeling 

Deployment


In [ ]:
#Now I am Going to import nessesary module 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
%matplotlib inline
from sklearn import tree
from sklearn import metrics
from sklearn.exceptions import FitFailedWarning
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')


In [ ]:
cale_data=pd.read_csv("calendar.csv")
list_data=pd.read_csv("listings.csv")
revi_data=pd.read_csv("reviews.csv")

In [ ]:
cale_data

In [ ]:
#Look at the head of calendar datasets
cale_data.head(20)

In [ ]:
#find missing values in calendar dataset
cale_data.isnull().sum()

In [ ]:
#to find % missing values in each columns
cale_data.isnull().sum()/cale_data.isnull().shape[0]

In [ ]:
cale_data.shape

In [ ]:
# cale_data.describe()

# Q1.My First Question is that how the price varies with the Date


Not only available days are stored in data, it seems to be stored not available days.

If the available values are f, the price values seems to be NaN.

The price values are stored as object, not integer. 
This is caused the value stored like $xx.xx, and it is necessary to transform this column.

In [ ]:
# Now Start to cleaning the calender datasets
def transformation(df):
    '''input:That has to be clean
    i am writing this function to convert the date object to data and converts the price 
    columns object to flot vlaues
    output: will be a clean dataframe'''
    df.dropna(inplace=True)
    df['date']=pd.to_datetime(df['date'])
    
    df['price']=df['price'].str.replace('$','')
    df['price']=df['price'].str.replace(",",'')
    df['price']=df['price'].astype(float)
    return df

In [ ]:
transformation(cale_data)

In [ ]:
#let's aggrigate price based on the date for analys this question
cale_data_df=cale_data.groupby("date")['price'].mean().reset_index()
cale_data_df

In [ ]:
#lets see how the price very over a period of time by plotting a line chart
plt.figure(figsize=(15,8))
plt.plot(cale_data_df['date'],cale_data_df['price'])
plt.title("Average price of a listing by date")
plt.xlabel("Average")
plt.ylabel("Average price")
plt.show()

In [ ]:
#for the same let's try to take  out the weekdays name from the data and the plot the price chart the weekdays
cale_data_df["day of the week"]=cale_data_df["date"].dt.day_name()
plt.figure(figsize=(15,8))
sns.boxplot(x='day of the week', y='price',data=cale_data_df ,palette="Blues", width=0.6)
plt.show()

Now we can understand that From the above visualisation,we figured out that the prices of 
the listings were maximum from July to September and were minimum from January to March

# Q2.How long is the period available for lending by rooms

Is there rooms which is available all for years? 

or almost rooms are available on spot like one day or one week?

Here, I want to know the trend in the outline of the data.

In [ ]:
#Now we are going to analys listing dataset
#fistly look at the head of the data
list_data.head()

In [ ]:
#Now find the missing values in the listing dataset
list_data.isnull().sum()

In [ ]:
# if I want to see the % missing values in each column, i have use below line
list_data.isnull().sum()/list_data.isnull().shape[0]

In [ ]:
#The Shape of listing data or no of rows and column in Listing 
list_data.shape

In [ ]:
#look at the number of id and row in the table
print("Num of listings: ", list_data.id.count())
print("Num of rows: ", list_data.shape[0])

From the above statement we can see that every row represents uniqe listings

In [ ]:
#now describe the data with 8 most important operations
list_data['review_scores_rating'].describe().reset_index()

In [ ]:
#lets see the histogram of review_scores_rating
plt.figure(figsize=(15,8))
plt.hist(list_data['review_scores_rating'].dropna().values,bins=80,color='b')
plt.grid()

This is very right skewed distribution.
The 75% or more values are 90 points. And the most common thing is 100 points.
I can say the low score listings are minolity.

In [ ]:
#now we are going to estemate outliers becouse values are very large
list_data = list_data[list_data['maximum_nights'] <= 1500]

plt.figure(figsize=(15, 8))
plt.hist(list_data.maximum_nights, bins=100, color='b')
plt.xlabel('maximum nights')
plt.ylabel('listings count')
plt.grid()

This was shown when investigating listing data. There were two groups in listings. It's a listing available at spots with a maximum nights less than a week and a listing of a sense of renting available for up to three years.

In [ ]:
#lets plot the histogram of listings
plt.figure(figsize=(15, 8))
plt.hist(list_data.price, bins=100, color='r')
plt.grid()

This is long tail distribution.
Almost values are from 0 to 200.

In [ ]:
revi_data.head()

In [ ]:
revi_data.isnull().sum()

In [ ]:
revi_data.isnull().sum()/revi_data.isnull().shape[0]

In [ ]:
revi_data.shape

In [ ]:
list_data.head()
list_data.columns

In [ ]:
print("sample 1: ", revi_data.comments.values[0], "\n")
print("sample 2: ", revi_data.comments.values[3])

In [ ]:
# convert date column's data type to date from object
review_df= revi_data.copy(deep=True)
review_df.date = pd.to_datetime(review_df.date)

review_df = review_df.groupby('date')['id'].count().reset_index()

# plot avg listings prices over time.
plt.figure(figsize=(15, 8))
plt.plot(review_df.date, review_df.id, color='b', linewidth=0.9)
plt.title("Number of reviews by date")
plt.xlabel('date')
plt.ylabel('number of reviews')
plt.grid()

It is little noisy, but we can see an increase in the number of Airbnb users. (and the date range is wide than calendar data)
And I realize it seems to have a peak at about the same time of each year.
So, let's use moving averages to smooth the graph.

In [ ]:
# create rolling mean column
review_df["rolling_mean_30"] = review_df.id.rolling(window=30).mean()

# plot avg listings prices over time.
plt.figure(figsize=(15, 8))
plt.plot(review_df.date, review_df.rolling_mean_30, color='b', linewidth=2.0)
plt.title("Number of reviews by date")
plt.xlabel('date')
plt.ylabel('number of reviews')
plt.grid()

I tried thirty days (about 1 month) window.
The graph became smooth and the trend became clear, and my belief that the peaks were in the same place became stronger.

In [ ]:
review_df["year"] = review_df.date.dt.year
years = review_df.year.unique()

for year in years:
    if year >= 2010 and year < 2016:
        year_df = review_df[review_df.year == year]
        max_value = year_df.rolling_mean_30.max()
        max_date = year_df[year_df.rolling_mean_30 == max_value].date.dt.date.values[0]
        print(year, max_date, np.round(max_value, 1))

In [ ]:
#For a further discussion, plot a scatter plot of maximum nights and minimum nights.
list_data["min_max_night_diff"] = list_data.maximum_nights - list_data.minimum_nights

plt.figure(figsize=(15, 8))
plt.plot(list_data.maximum_nights, list_data.minimum_nights, color='b', marker='o', linewidth=0, alpha=0.25)
plt.xlabel('maximum nights')
plt.ylabel('minimum nights')
plt.grid()

From here, it can be seen that the minimum nights is almost constant regardless of the maximum nights. In other words, it can be seen that listings with a long maximum nights are not rented exclusively for rental, but are widely handled from spot use to long-term stay.

# Q3.Is there a busy season?

In [ ]:
review_df.columns

In [ ]:
review_df2 = review_df[review_df.year == 2015]

plt.figure(figsize=(15, 8))
plt.plot(review_df2.date, review_df2.rolling_mean_30, color='b', linewidth=2.0)
plt.title("Number of reviews by date")
plt.grid()

# Q4.Are there any trends of popular rooms?

First, we need to define the target variable 'Popular of listings'. I thought this could be defined as follows.

[Actual number of times rent] / ([Available days from 2016 to 2017] * (2017 - [Year of the listings open]))

Since the number of times the listings are actually rent is considered to be proportional to the number of days the property is available, it needs to be scaled, and Furthermore, since the number is considered to increase the earlier the listings is released to the public, we need to scale there as well.

# Data Preparation

In [ ]:
prepare_df = list_data.copy(deep=True)

In [ ]:
# check null count
df_length = prepare_df.shape[0]

for col in prepare_df.columns:
    null_count = prepare_df[col].isnull().sum()
    if null_count == 0:
        continue
        
    null_ratio = np.round(null_count/df_length * 100, 2)
    print("{} has {} null values ({}%)".format(col, null_count, null_ratio))

As you can see, it seems that there are columns 90% or more of the values are null, but most of the columns have between 0-30% of null_ratio. Therefore, here, I decided to excluded from analysis the columns with 30% or more null ratio

In [ ]:
# detect need drop columns
drop_cols = [col for col in prepare_df.columns if prepare_df[col].isnull().sum()/df_length >= 0.3]

# drop null
prepare_df.drop(drop_cols, axis=1, inplace=True)
prepare_df.dropna(subset=['host_since'], inplace=True)

# check after
for col in prepare_df.columns:
    null_count = prepare_df[col].isnull().sum()
    if null_count == 0:
        continue
        
    null_ratio = np.round(null_count/df_length * 100, 2)
    print("{} has {} null values ({}%)".format(col, null_count, null_ratio))


Good.
Next, we will delete the column that seems not to be a feature of ease of borrowing. Since we do not include natural language processing this time, we will also delete comment-based columns.

In [ ]:
drop_cols = ['listing_url', 'scrape_id', 'last_scraped', 'name', 'summary', 'space', 'description', 'neighborhood_overview',
                'transit', 'medium_url', 'picture_url', 'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_about', 'host_thumbnail_url',
                'host_picture_url', 'street', 'city', 'state', 'zipcode', 'market', 'smart_location', 'country_code', 'country', 'latitude', 'longitude',
                'calendar_updated', 'calendar_last_scraped', 'first_review', 'last_review', 'amenities', 'host_verifications']

prepare_df.drop(drop_cols, axis=1, inplace=True)

In [ ]:
prepare_df.columns

Furthermore, delete columns that have only a single value, because the feature value does not make sense.

In [ ]:
drop_cols = []
for col in prepare_df.columns:
    if prepare_df[col].nunique() == 1:
        drop_cols.append(col)
        
prepare_df.drop(drop_cols, axis=1, inplace=True)
prepare_df.columns

Good. Now I almost finished remove columns that is not used for anallysis. So next, I create target valuable.

In [ ]:
cale_data

In [ ]:
# available days count each listings
listing_avalilable =cale_data.groupby('listing_id')['price'].count().reset_index()
listing_avalilable.columns = ["id", "available_count"]

# merge
prepare_df = prepare_df.merge(listing_avalilable, how='left', on='id')

# create target column
prepare_df['host_since_year'] = pd.to_datetime(prepare_df['host_since']).dt.year
prepare_df["easily_accomodated"] = prepare_df.accommodates / (prepare_df.available_count+1) / (2017 - prepare_df.host_since_year)

In [ ]:
print("Before: {} columns".format(prepare_df.shape[1]))

drop_cols = ['host_since', 'accommodates', 'availability_30', 'availability_60', 'availability_90', 'availability_365',
                'number_of_reviews', 'review_scores_rating', 'available_count', 'reviews_per_month', 'host_since_year', 'review_scores_value']

prepare_df.drop(drop_cols, axis=1, inplace=True)
print("After: {} columns".format(prepare_df.shape[1]))

In [ ]:
# convert true or false value to 1 or 0
dummy_cols = ['host_is_superhost', 'require_guest_phone_verification', 'require_guest_profile_picture', 'instant_bookable', 
              'host_has_profile_pic', 'host_identity_verified', 'is_location_exact']

for col in dummy_cols:
    prepare_df[col] = prepare_df[col].map(lambda x: 1 if x == 't' else 0)

# create dummy valuables
dummy_cols = ['host_location', 'host_neighbourhood', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed',
             'property_type', 'room_type', 'bed_type', 'cancellation_policy', 'host_response_time']

prepare_df = pd.get_dummies(prepare_df, columns=dummy_cols, dummy_na=True)


In [ ]:
df_length = prepare_df.shape[0]

for col in prepare_df.columns:
    null_count = prepare_df[col].isnull().sum()
    if null_count == 0:
        continue
        
    null_ratio = np.round(null_count/df_length * 100, 2)
    print("{} has {} null values ({}%)".format(col, null_count, null_ratio))

Next, handle any remaining null values.

In [ ]:
prepare_df["is_thumbnail_setted"] = 1 - prepare_df.thumbnail_url.isnull()
prepare_df.drop('thumbnail_url', axis=1, inplace=True)
prepare_df.host_response_rate = prepare_df.host_response_rate.fillna('0%').map(lambda x: float(x[:-1]))
prepare_df.host_acceptance_rate = prepare_df.host_acceptance_rate.fillna('0%').map(lambda x: float(x[:-1]))
prepare_df.bathrooms.fillna(0, inplace=True)
prepare_df.bedrooms.fillna(0, inplace=True)
prepare_df.beds.fillna(0, inplace=True)
prepare_df.cleaning_fee.fillna('$0', inplace=True)
prepare_df.review_scores_accuracy.fillna(0, inplace=True)
prepare_df.review_scores_cleanliness.fillna(0, inplace=True)
prepare_df.review_scores_checkin.fillna(0, inplace=True)
prepare_df.review_scores_communication.fillna(0, inplace=True)
prepare_df.review_scores_location.fillna(0, inplace=True)


Finally, since the value to be acted as a numeric value is recognized as a string, so let's convert it.

In [ ]:
for col in prepare_df.columns:
    if prepare_df[col].dtypes == 'object':
        print(col)

In [ ]:
prepare_df.price = prepare_df.price.map(lambda x: float(x[1:].replace(',', '')))
prepare_df.cleaning_fee = prepare_df.cleaning_fee.map(lambda x: float(x[1:].replace(',', '')))
prepare_df.extra_people = prepare_df.extra_people.map(lambda x: float(x[1:].replace(',', '')))

# Conclusion

The prices of the listings are maximum from July to September and are minimum from January to March which is understood as July to September is the best time to visit Seattle because of the nice weather and therefore the prices are highest during that time because of the high demand.
The average rating score of Arbor Heights is maximum followed by Riverview and Southeast Magnolia.The ratings of a listing is influenced by whether a host is a superhost or not,the host’s response rate,the size of the listings,the number of amenities provided in the listing and also on its price.
Price of a listing depends on its neighborhood,size(number of bedrooms,number of bathrooms),type of the property i.e. if it is a bungalow,the price is higher than the other property and on whether the host is a superhost or not.